In [165]:
import pandas as pd
import requests
import folium
import polyline

## DF Prueba

df_destinos = pd.DataFrame(
    {'Id': ['1', '2', '3', '4', '5','6','7', '8','9','10'],
     'Latitud': ["19.343881","19.332822","19.347733","19.363256","19.367041",
                 "19.350522","19.355939","19.385408","19.367081","19.332822"],
     
     'Longitud': ["-99.280132","-99.244732","-99.232666","-99.287127","-99.259104",
                  "-99.300560","-99.283770","-99.258914","-99.298912","-99.274842"]})

df_origen = pd.DataFrame(
    {'Id': ['origen'],
          'Latitud': ["19.358923"],
     'Longitud': ["-99.269832"]
    }
     )

df_opt = df_origen.append(df_destinos)



In [166]:
def generador_destinos(df_o):
    """
    df_opt = data frame con información de la latitud y longitud
    destinos = regresamos el formato para utilizar en la solicitud de ruta optima
    """
    
    df_o["loc"] = df_o["Longitud"]+ "," + df_o["Latitud"]
    List = df_o["loc"]
    seperador = ';'
    destinos = seperador.join(List)
    return destinos


def opt_ruta(destinos):
    """
    destinos = lista de ruta a optimizar
    data = resultados
    """
    
    # url = "http://65.22.168.65:9080/trip/v1/driving/" # servidor
    url = 'http://router.project-osrm.org/trip/v1/driving/'
    params = dict(
            source= 'any',
            destination='any'
        )
    response = requests.get(url=url+destinos, params=params)
    data = response.json()
    return data



In [167]:
## Solicitamos info
destinos = generador_destinos(df_opt)
data = opt_ruta(destinos)
df = pd.DataFrame.from_dict(data["waypoints"])
location = pd.DataFrame(df.location.tolist(), columns=['X', 'Y'])
df = pd.concat([df, location], axis=1)

In [168]:
### waypoint_index nos regresa el orden de asignación.
df

,distance,hint,location,name,trips_index,waypoint_index,X,Y
0,4.994223,GOWUhf___38MAAAA4AAAAG8AAAD0AAAA71NiQcw0ckL9X_...,"[-99.269802, 19.358888]",Paseo de los Arquitectos,0,0,-99.269802,19.358888
1,10.480328,MjQDgFg0kYUuAAAAJQAAAEYBAAAdAQAApOUPQVU6_0DUuX...,"[-99.280035, 19.343903]",Calle Fresnos,0,10,-99.280035,19.343903
2,6.942790,cPyQhc_8kIUbAAAAEgAAAKUAAACKAAAA3fEzQTNf8UBc2o...,"[-99.244742, 19.33276]",Calle Héctor Sarabia,0,8,-99.244742,19.332760
3,10.864745,Mgughf___39aAAAApwAAAAAAAAAAAAAAnw8qQkVitEEAAA...,"[-99.23264, 19.347638]",Calzada de las Águilas,0,7,-99.232640,19.347638
4,3.286679,FoiJgIOIiYBAAAAAjQAAAAAAAABmAQAA_eGqQRSz90EAAA...,"[-99.287096, 19.36326]",San José de los Cedros,0,3,-99.287096,19.363260
5,17.253316,G_KUhf___396AAAADgEAALEAAABAAAAAqQhZQvynDkKlhp...,"[-99.259216, 19.366927]",Calle Antonio Dovalí Jaime,0,1,-99.259216,19.366927
6,4.014093,4jkXgP___38-AAAAXgEAAL8AAAAAAAAA59v5QW2V0UIUKb...,"[-99.30055, 19.350487]",,0,5,-99.300550,19.350487
7,5.480873,IuiUhf___38BAAAAPQAAANAAAADfAAAAAY2XP3tgs0FjN0...,"[-99.283721, 19.355956]",Vasco de Quiroga,0,6,-99.283721,19.355956
8,3.843233,k-uUhf___38UAAAA6wAAANEAAAB6BAAAiR-wQdA9ikLhk1...,"[-99.258891, 19.385381]",Paseo de los Laureles,0,2,-99.258891,19.385381
9,22.167797,qTOVha0zlYUAAAAAbwAAAAAAAAAAAAAAAAAAAI1OOUIAAA...,"[-99.298781, 19.366924]",Cerrada Salvador Arreola,0,4,-99.298781,19.366924


In [169]:
## visualizamos resultados
from folium.features import DivIcon

# Zona
Origen = (float(df_origen.Latitud[0]),float(df_origen.Longitud[0]))
map = folium.Map(location=Origen, zoom_start=12)

# Visualizamos  asignación
for each in df.iterrows():
    folium.Marker(
        location = [each[1]['Y'],each[1]['X']],
        clustered_marker = True,
        popup=each[1]["waypoint_index"],
        icon= DivIcon(
        icon_size= (150,36),
        icon_anchor=(0,0),
        html='<div style="font-size: 24pt">%s</div>' %each[1]["waypoint_index"],
        )
        # icon=folium.Icon(icon='home', color='green')
    ).add_to(map)
  
folium.Marker(
        location=[df['Y'][0],df['X'][0]],
        icon=folium.Icon(icon='play', 
                         color='red')).add_to(map)

# routes = polyline.decode(data['trips'][0]['geometry'])
# folium.PolyLine(
#         routes,
#        weight=8,
#        color='blue',
#        opacity=0.6
#    ).add_to(map)

map.save('asignacion_popups.html')
display(map)